In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain.tools import tool
from typing import List, Dict
import dotenv

In [29]:
dotenv.load_dotenv()

True

In [2]:
working_directory = "test"

In [3]:
toolkit = FileManagementToolkit(root_dir=working_directory)

In [5]:
available_tools = toolkit.get_tools()

In [6]:
print("[사용 가능한 파일 관리 도구들]")
for tool in available_tools:
    print(f"- {tool.name}: {tool.description}")

[사용 가능한 파일 관리 도구들]
- copy_file: Create a copy of a file in a specified location
- file_delete: Delete a file
- file_search: Recursively search for files in a subdirectory that match the regex pattern
- move_file: Move or rename a file from one location to another
- read_file: Read file from disk
- write_file: Write file to disk
- list_directory: List files and directories in a specified folder


In [7]:
tools = FileManagementToolkit(
    root_dir=working_directory,
    selected_tools=["read_file", "file_delete", "write_file", "list_directory"],
).get_tools()

tools

[ReadFileTool(root_dir='test'),
 DeleteFileTool(root_dir='test'),
 WriteFileTool(root_dir='test'),
 ListDirectoryTool(root_dir='test')]

In [8]:
read_tool, delete_tool, write_tool, list_tool = tools

# 파일 쓰기
write_tool.invoke({"file_path": "example.txt", "text": "Hello World!"})

'File written successfully to example.txt.'

In [12]:
write_tool.invoke({"file_path": "example23.txt", "text": "GoodBye World!"})

'File written successfully to example23.txt.'

In [15]:
from IPython.display import Markdown

In [17]:
result = list_tool.invoke({})

In [63]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

In [64]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 강력한 어시스턴트입니다. 어떤 질문에도 훌륭한 답변을 줄 수 있습니다. "
            "Make sure to use the `file_management` tool to manage files. ",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [65]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [66]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)

In [67]:
store = {}

# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

In [68]:
# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "query"
    input_messages_key="query",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [69]:
result = agent_with_chat_history.invoke(
        {"query":"""디렉토리에 Aiffel10.txt라는 파일을 생성하고 내부에는 '안녕하세요. 방승욱입니다.' 라고 입력해줘"""
        },
        config = {"configurable":{"session_id": "a1"}})

In [70]:
result2 = agent_with_chat_history.stream(
        {"query":"""디렉토리에 Aiffel10.txt라는 파일을 생성하고 내부에는 '안녕하세요. 방승욱입니다.' 라고 입력해줘"""
        },
        config = {"configurable":{"session_id": "a2"}})

In [71]:
result

{'query': "디렉토리에 Aiffel10.txt라는 파일을 생성하고 내부에는 '안녕하세요. 방승욱입니다.' 라고 입력해줘",
 'chat_history': [],
 'output': "Aiffel10.txt 파일이 생성되었고, 내부에 '안녕하세요. 방승욱입니다.'라는 내용이 입력되었습니다."}

In [72]:
result2

<generator object RunnableBindingBase.stream at 0x0000024C5F795D20>

In [61]:
store

{'a1': InMemoryChatMessageHistory(messages=[])}

In [73]:
for step in result2:
    print(step)

{'actions': [ToolAgentAction(tool='write_file', tool_input={'file_path': 'Aiffel10.txt', 'text': '안녕하세요. 방승욱입니다.'}, log="\nInvoking: `write_file` with `{'file_path': 'Aiffel10.txt', 'text': '안녕하세요. 방승욱입니다.'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_c4tfgFWyepuUQQrXLAwKJ8iH', 'function': {'arguments': '{"file_path":"Aiffel10.txt","text":"안녕하세요. 방승욱입니다."}', 'name': 'write_file'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306'}, id='run-e374720d-f881-4d95-a926-7284437a555e', tool_calls=[{'name': 'write_file', 'args': {'file_path': 'Aiffel10.txt', 'text': '안녕하세요. 방승욱입니다.'}, 'id': 'call_c4tfgFWyepuUQQrXLAwKJ8iH', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'write_file', 'args': '{"file_path":"Aiffel10.txt","text":"안녕하세요. 방승욱입니다."}', 'id': 'call_c4tfgFWyepuUQQrXLAwKJ8iH', 'index': 0, 'type': 'tool_call_chunk'}])], too

In [74]:
store

{'a1': InMemoryChatMessageHistory(messages=[HumanMessage(content="디렉토리에 Aiffel10.txt라는 파일을 생성하고 내부에는 '안녕하세요. 방승욱입니다.' 라고 입력해줘", additional_kwargs={}, response_metadata={}), AIMessage(content="Aiffel10.txt 파일이 생성되었고, 내부에 '안녕하세요. 방승욱입니다.'라는 내용이 입력되었습니다.", additional_kwargs={}, response_metadata={})]),
 'a2': InMemoryChatMessageHistory(messages=[HumanMessage(content="디렉토리에 Aiffel10.txt라는 파일을 생성하고 내부에는 '안녕하세요. 방승욱입니다.' 라고 입력해줘", additional_kwargs={}, response_metadata={}), AIMessage(content="Aiffel10.txt 파일이 생성되었고, 내부에 '안녕하세요. 방승욱입니다.'라는 내용이 입력되었습니다.", additional_kwargs={}, response_metadata={})])}